# import modules & load files

import modules

In [9]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import warnings
warnings.filterwarnings('ignore')
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1


load files

In [10]:
# strike price set
CTB09sp = pd.read_csv('./target_price_set/CTB09sp.csv', index_col=0)
CTB15sp = pd.read_csv('./target_price_set/CTB15sp.csv', index_col=0)
PTB09sp = pd.read_csv('./target_price_set/PTB09sp.csv', index_col=0)
PTB15sp = pd.read_csv('./target_price_set/PTB15sp.csv', index_col=0)

In [11]:
CTB09sp

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,5.44,NaN,NaN,NaN,7.03,3.92,2.74,1.80,1.12
2023-10-30,4.56,NaN,NaN,NaN,6.08,3.32,2.31,1.54,0.98
2023-10-27,5.21,NaN,NaN,NaN,6.74,3.90,2.81,1.93,1.27
2023-10-26,3.22,9.14,7.40,5.79,4.39,2.25,1.51,0.97,0.60
2023-10-25,5.31,12.95,10.50,9.14,6.94,4.00,2.87,1.98,1.30
2023-10-24,5.77,13.25,11.00,9.09,7.33,4.43,3.28,2.32,1.57
2023-10-23,4.97,12.30,9.65,8.15,6.38,3.78,2.76,1.93,1.30
2023-10-20,3.59,9.31,7.55,6.04,4.70,2.64,1.86,1.27,0.84
2023-10-19,3.49,9.09,7.50,6.03,4.65,2.53,1.76,1.18,0.75


In [12]:
CTB15sp

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,2.97,NaN,NaN,NaN,4.21,1.98,1.25,0.74,0.41
2023-10-30,5.40,NaN,NaN,NaN,7.16,3.99,2.82,1.89,1.20
2023-10-27,5.15,NaN,NaN,NaN,6.65,3.82,2.72,1.86,1.20
2023-10-26,2.23,7.15,5.64,4.29,3.15,1.52,0.99,0.63,0.39
2023-10-25,4.29,11.00,9.08,7.30,5.68,3.12,2.16,1.43,0.89
2023-10-24,7.11,14.30,12.60,10.60,8.66,5.58,4.25,3.10,2.17
2023-10-23,4.44,10.85,9.00,7.39,5.77,3.30,2.36,1.62,1.06
2023-10-20,3.46,9.17,7.40,5.89,4.58,2.52,1.77,1.20,0.78
2023-10-19,3.28,8.90,7.20,5.70,4.40,2.37,1.64,1.08,0.69


In [13]:
PTB09sp

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,2.76,NaN,NaN,NaN,1.95,3.80,5.12,6.65,8.50
2023-10-30,4.15,NaN,NaN,NaN,3.13,5.40,6.86,8.60,10.30
2023-10-27,4.18,NaN,NaN,NaN,3.21,5.36,6.64,8.38,10.30
2023-10-26,6.41,2.34,3.06,3.95,5.05,7.93,9.57,11.20,13.75
2023-10-25,4.04,1.26,1.72,2.32,3.09,5.19,6.54,8.00,9.47
2023-10-24,4.31,1.52,2.00,2.62,3.39,5.46,6.80,8.16,10.10
2023-10-23,5.63,2.24,2.86,3.61,4.53,6.90,8.60,10.30,12.15
2023-10-20,7.80,3.47,4.31,5.27,6.47,9.35,11.05,13.05,14.90
2023-10-19,6.95,2.77,3.54,4.49,5.63,8.54,10.45,12.30,14.35


In [14]:
PTB15sp

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,4.76,NaN,NaN,NaN,3.53,6.27,8.03,10.05,12.35
2023-10-30,3.19,NaN,NaN,NaN,2.34,4.26,5.50,7.09,8.50
2023-10-27,3.99,NaN,NaN,NaN,3.04,5.14,6.56,8.23,10.05
2023-10-26,8.63,3.56,4.52,5.69,7.04,10.35,12.25,14.35,16.75
2023-10-25,4.79,1.55,2.10,2.81,3.70,6.11,7.63,9.48,11.40
2023-10-24,3.33,1.08,1.45,1.93,2.55,4.28,5.42,6.84,8.52
2023-10-23,5.85,2.30,2.95,3.74,4.70,7.24,8.78,10.50,12.40
2023-10-20,7.89,3.47,4.31,5.32,6.47,9.35,11.15,13.10,14.85
2023-10-19,7.57,3.18,4.00,5.00,6.17,9.15,10.90,12.95,15.05


In [22]:
# real label
VKOSPI_Label = pd.read_csv('./VKOSPI.csv', index_col=0)
VKOSPI_Label[['cost', 'revenue', 'profit']] = 0
VKOSPI_Label.set_index('날짜', inplace=True)
VKOSPI_Label

,VKOSPI_Label,cost,revenue,profit
날짜,,,,
2023-10-10,1,0,0,0
2023-10-11,0,0,0,0
2023-10-12,0,0,0,0
2023-10-13,0,0,0,0
2023-10-16,1,0,0,0
2023-10-17,0,0,0,0
2023-10-18,0,0,0,0
2023-10-19,1,0,0,0
2023-10-20,1,0,0,0


In [ ]:
# predict labels
output = pd.read_csv("./result_quarter_output.csv", index_col=0)
labels = output.iloc[:,-7:]
labels[['cost', 'revenue', 'profit']] = 0

In [ ]:
labels

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM,cost,revenue,profit
2023-10-10,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-11,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-12,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-13,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-16,0.0,0.0,0.0,0.0,0.0,1,1.0,0,0,0
2023-10-17,0.0,1.0,1.0,1.0,1.0,0,0.0,0,0,0
2023-10-18,1.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-19,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0
2023-10-20,0.0,1.0,1.0,1.0,1.0,1,1.0,0,0,0
2023-10-23,1.0,1.0,0.0,1.0,1.0,1,1.0,0,0,0


In [ ]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   VKOSPI_Label        19 non-null     float64
 1   LogisticRegression  19 non-null     float64
 2   DecisionTree        19 non-null     float64
 3   SVC                 19 non-null     float64
 4   RandomForest        19 non-null     float64
 5   XGB                 19 non-null     int64  
 6   LGBM                19 non-null     float64
 7   cost                19 non-null     int64  
 8   revenue             19 non-null     int64  
 9   profit              19 non-null     int64  
dtypes: float64(6), int64(4)
memory usage: 1.6+ KB


In [ ]:
labels.describe()

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM,cost,revenue,profit
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.0,19.0
mean,0.526316,0.263158,0.368421,0.210526,0.263158,0.315789,0.263158,0.0,0.0,0.0
std,0.512989,0.452414,0.495595,0.418854,0.452414,0.477567,0.452414,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,1.000000,0.500000,1.000000,0.000000,0.500000,1.000000,0.500000,0.0,0.0,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0


In [ ]:
VKOSPI_Label = labels[['VKOSPI_Label', 'cost', 'revenue', 'profit']]
LogisticRegression = labels[['LogisticRegression', 'cost', 'revenue', 'profit']]
DecisionTree = labels[['DecisionTree', 'cost', 'revenue', 'profit']]
SVC = labels[['SVC', 'cost', 'revenue', 'profit']]
RandomForest = labels[['RandomForest', 'cost', 'revenue', 'profit']]
XGB = labels[['XGB', 'cost', 'revenue', 'profit']]
LGBM = labels[['LGBM', 'cost', 'revenue', 'profit']]

In [ ]:
VKOSPI_Label.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   VKOSPI_Label  19 non-null     float64
 1   cost          19 non-null     int64  
 2   revenue       19 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 608.0+ bytes


매매전략별 수익률 계산
=====
if label == -1: # 하락 → 양매도 *-1
    인덱스(시간) 일치
    case 1: 스트래들매도
    CTB09sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    PTB09sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    CTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매수(청산)함
    
    case 2: 스트랭글
    CTB09sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB09sp에서 인덱스(시간) 기준행사가-4 P매도
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매수
    수익률 계산 = 매수 값 / 매도 값
    수익금 계산 = 매수 값 - 매도 값

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매도 *2
            기준행사가-1 P매수 
            기준행사가+1 P매수
        case 3-2:
            기준행사가 C매도 *2
            기준행사가-1 C매수
            기준행사가+1 C매수
    
elif Label == 0: # 보합
    관망 how to code
    
elif label == 1: # 상승 → 양매수
    인덱스(시간) 일치
    case 1: 스트래들매수
    CTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    PTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    CTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    수익률 계산 = 매도 값 / 매수 값
    수익금 계산 = 매도 값 - 매수 값

    case 2: 스트랭글매수
    CTB9sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB9sp에서 인덱스(시간) 기준행사가-4 P매수
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매도

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매수 *2
            기준행사가-1 P매도
            기준행사가+1 P매도
        case 3-2:
            기준행사가 C매수 *2
            기준행사가-1 C매도
            기준행사가+1 C매도

# 양매도 스트랭글과 양매수 스트래들

실제 변동

In [ ]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i]
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB09sp['S+4'].loc[d]
            cost += PTB09sp['S-4'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost # 9시 값 합
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue # 15시 값 합
            VKOSPI_Label['profit'].loc[d] = cost - revenue
            print(d, VKOSPI_Label.loc[d, 'VKOSPI_Label'],'cost:', cost, 'revenue:', revenue, 'profit:', VKOSPI_Label['profit'].loc[d])
        except:     
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan 
            print(d, np.nan)
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
            VKOSPI_Label['profit'].loc[d] = revenue - cost
            print(d, VKOSPI_Label.loc[d, 'VKOSPI_Label'], 'cost:', cost, 'revenue:', revenue, 'profit:', VKOSPI_Label['profit'].loc[d])
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
            print(d, np.nan)

2023-10-10 nan
2023-10-11 nan
2023-10-12 nan
2023-10-13 nan
2023-10-16 nan
2023-10-17 nan
2023-10-18 nan
2023-10-19 nan
2023-10-20 nan
2023-10-23 nan
2023-10-24 nan
2023-10-25 nan
2023-10-26 nan
2023-10-27 nan
2023-10-30 nan
2023-10-31 nan
2023-11-01 nan
2023-11-02 nan
2023-11-03 nan


In [ ]:
# print(VKOSPI_Label['profit'].sum())

In [ ]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i]
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB09sp['S+4'].loc[d]
            cost += PTB09sp['S-4'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost # 9시 값 합
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue # 15시 값 합
            VKOSPI_Label['profit'].loc[d] = cost - revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
            VKOSPI_Label['profit'].loc[d] = revenue - cost
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan

In [ ]:
# VKOSPI_Label['profit'] = VKOSPI_Label['revenue'] - VKOSPI_Label['cost']
print(VKOSPI_Label)
print(VKOSPI_Label['profit'].sum())

            VKOSPI_Label  cost  revenue
2023-10-10           1.0   NaN      NaN
2023-10-11           1.0   NaN      NaN
2023-10-12           1.0   NaN      NaN
2023-10-13           0.0   NaN      NaN
2023-10-16           0.0   NaN      NaN
2023-10-17           0.0   NaN      NaN
2023-10-18           1.0   NaN      NaN
2023-10-19           0.0   NaN      NaN
2023-10-20           0.0   NaN      NaN
2023-10-23           1.0   NaN      NaN
2023-10-24           1.0   NaN      NaN
2023-10-25           0.0   NaN      NaN
2023-10-26           0.0   NaN      NaN
2023-10-27           1.0   NaN      NaN
2023-10-30           1.0   NaN      NaN
2023-10-31           1.0   NaN      NaN
2023-11-01           0.0   NaN      NaN
2023-11-02           1.0   NaN      NaN
2023-11-03           0.0   NaN      NaN


KeyError: 'profit'

LogisticRegression

In [ ]:
for i in range(len(LogisticRegression.index)):
    d = LogisticRegression.index[i]
    if LogisticRegression.loc[d, 'LogisticRegression'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB09sp['S+4'].loc[d]
            cost += PTB09sp['S-4'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
            LogisticRegression['profit'].loc[d] = cost - revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
            LogisticRegression['profit'].loc[d] = revenue - cost
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
# LogisticRegression['profit'] = LogisticRegression['revenue'] - LogisticRegression['cost']
print(LogisticRegression)
print(LogisticRegression['profit'].sum())

            LogisticRegression   cost  revenue  profit
2023-10-10                 0.0    NaN      NaN     NaN
2023-10-11                 0.0    NaN      NaN     NaN
2023-10-12                 0.0    NaN      NaN     NaN
2023-10-13                 0.0   3.74     3.62    0.12
2023-10-16                 0.0   3.57     3.60   -0.03
2023-10-17                 1.0  10.74    11.15    0.41
2023-10-18                 0.0   3.20     3.34   -0.14
2023-10-19                 0.0   3.52     3.87   -0.35
2023-10-20                 1.0  11.39    11.35   -0.04
2023-10-23                 1.0  10.60    10.29   -0.31
2023-10-24                 0.0   3.09     3.25   -0.16
2023-10-25                 0.0   2.56     2.44    0.12
2023-10-26                 0.0   2.94     3.95   -1.01
2023-10-27                 1.0   9.39     9.14   -0.25
2023-10-30                 0.0    NaN      NaN     NaN
2023-10-31                 0.0    NaN      NaN     NaN
2023-11-01                 1.0    NaN      NaN    0.00
2023-11-02

DecisionTree

In [ ]:
for i in range(len(DecisionTree.index)):
    d = DecisionTree.index[i]
    if DecisionTree.loc[d, 'DecisionTree'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB09sp['S+4'].loc[d]
            cost += PTB09sp['S-4'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
            DecisionTree['profit'] = DecisionTree['cost'] - DecisionTree['revenue']
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
            DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan

In [ ]:
DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
print(DecisionTree)
print(DecisionTree['profit'].sum())

            DecisionTree  cost  revenue  profit
2023-10-10           0.0   NaN      NaN     NaN
2023-10-11           0.0   NaN      NaN     NaN
2023-10-12           0.0   NaN      NaN     NaN
2023-10-13           0.0   NaN      NaN     NaN
2023-10-16           0.0   NaN      NaN     NaN
2023-10-17           1.0   NaN      NaN     NaN
2023-10-18           0.0   NaN      NaN     NaN
2023-10-19           0.0   NaN      NaN     NaN
2023-10-20           1.0   NaN      NaN     NaN
2023-10-23           0.0   NaN      NaN     NaN
2023-10-24           1.0   NaN      NaN     NaN
2023-10-25           0.0   NaN      NaN     NaN
2023-10-26           1.0   NaN      NaN     NaN
2023-10-27           1.0   NaN      NaN     NaN
2023-10-30           1.0   NaN      NaN     NaN
2023-10-31           0.0   NaN      NaN     NaN
2023-11-01           1.0   NaN      NaN     NaN
2023-11-02           0.0   NaN      NaN     NaN
2023-11-03           0.0   NaN      NaN     NaN
0.0


SVC

In [ ]:
for i in range(len(SVC.index)):
    d = SVC.index[i]
    if SVC.loc[d, 'SVC'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
SVC['profit'] = SVC['revenue'] - SVC['cost']
print(SVC)
print(SVC['profit'].sum())

            SVC   cost  revenue  profit
2023-10-10  0.0    NaN      NaN     NaN
2023-10-11  0.0    NaN      NaN     NaN
2023-10-12  0.0    NaN      NaN     NaN
2023-10-13  0.0   3.74     3.62   -0.12
2023-10-16  0.0   3.57     3.60    0.03
2023-10-17  1.0  10.74    11.15    0.41
2023-10-18  0.0   3.20     3.34    0.14
2023-10-19  0.0   3.52     3.87    0.35
2023-10-20  1.0  11.39    11.35   -0.04
2023-10-23  1.0  10.60    10.29   -0.31
2023-10-24  0.0   3.09     3.25    0.16
2023-10-25  0.0   2.56     2.44   -0.12
2023-10-26  0.0   2.94     3.95    1.01
2023-10-27  1.0   9.39     9.14   -0.25
2023-10-30  0.0    NaN      NaN     NaN
2023-10-31  0.0    NaN      NaN     NaN
2023-11-01  0.0    NaN      NaN     NaN
2023-11-02  0.0    NaN      NaN     NaN
2023-11-03  0.0    NaN      NaN     NaN
1.259999999999998


RandomForest

In [ ]:
for i in range(len(RandomForest.index)):
    d = RandomForest.index[i]
    if RandomForest.loc[d, 'RandomForest'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
RandomForest['profit'] = RandomForest['revenue'] - RandomForest['cost']
print(RandomForest)
print(RandomForest['profit'].sum())

            RandomForest   cost  revenue  profit
2023-10-10           0.0    NaN      NaN     NaN
2023-10-11           0.0    NaN      NaN     NaN
2023-10-12           0.0    NaN      NaN     NaN
2023-10-13           0.0   3.74     3.62   -0.12
2023-10-16           0.0   3.57     3.60    0.03
2023-10-17           1.0  10.74    11.15    0.41
2023-10-18           0.0   3.20     3.34    0.14
2023-10-19           0.0   3.52     3.87    0.35
2023-10-20           1.0  11.39    11.35   -0.04
2023-10-23           1.0  10.60    10.29   -0.31
2023-10-24           0.0   3.09     3.25    0.16
2023-10-25           0.0   2.56     2.44   -0.12
2023-10-26           0.0   2.94     3.95    1.01
2023-10-27           1.0   9.39     9.14   -0.25
2023-10-30           0.0    NaN      NaN     NaN
2023-10-31           0.0    NaN      NaN     NaN
2023-11-01           1.0    NaN      NaN     NaN
2023-11-02           0.0    NaN      NaN     NaN
2023-11-03           0.0    NaN      NaN     NaN
1.259999999999998


XGB

In [ ]:
for i in range(len(XGB.index)):
    d = XGB.index[i]
    if XGB.loc[d, 'XGB'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
XGB['profit'] = XGB['revenue'] - XGB['cost']
print(XGB)
print(XGB['profit'].sum())

            XGB   cost  revenue  profit
2023-10-10    0    NaN      NaN     NaN
2023-10-11    0    NaN      NaN     NaN
2023-10-12    0    NaN      NaN     NaN
2023-10-13    0   3.74     3.62   -0.12
2023-10-16    1  10.66    10.66    0.00
2023-10-17    0   3.59     3.79    0.20
2023-10-18    0   3.20     3.34    0.14
2023-10-19    0   3.52     3.87    0.35
2023-10-20    1  11.39    11.35   -0.04
2023-10-23    1  10.60    10.29   -0.31
2023-10-24    0   3.09     3.25    0.16
2023-10-25    0   2.56     2.44   -0.12
2023-10-26    1   9.63    10.86    1.23
2023-10-27    1   9.39     9.14   -0.25
2023-10-30    0    NaN      NaN     NaN
2023-10-31    0    NaN      NaN     NaN
2023-11-01    1    NaN      NaN     NaN
2023-11-02    0    NaN      NaN     NaN
2023-11-03    0    NaN      NaN     NaN
1.2399999999999989


LGBM

In [ ]:
for i in range(len(LGBM.index)):
    d = LGBM.index[i]
    if LGBM.loc[d, 'LGBM'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
LGBM['profit'] = LGBM['revenue'] - LGBM['cost']
print(LGBM)
print(LGBM['profit'].sum())

            LGBM   cost  revenue  profit
2023-10-10   0.0    NaN      NaN     NaN
2023-10-11   0.0    NaN      NaN     NaN
2023-10-12   0.0    NaN      NaN     NaN
2023-10-13   0.0   3.74     3.62   -0.12
2023-10-16   1.0  10.66    10.66    0.00
2023-10-17   0.0   3.59     3.79    0.20
2023-10-18   0.0   3.20     3.34    0.14
2023-10-19   0.0   3.52     3.87    0.35
2023-10-20   1.0  11.39    11.35   -0.04
2023-10-23   1.0  10.60    10.29   -0.31
2023-10-24   0.0   3.09     3.25    0.16
2023-10-25   0.0   2.56     2.44   -0.12
2023-10-26   1.0   9.63    10.86    1.23
2023-10-27   1.0   9.39     9.14   -0.25
2023-10-30   0.0    NaN      NaN     NaN
2023-10-31   0.0    NaN      NaN     NaN
2023-11-01   0.0    NaN      NaN     NaN
2023-11-02   0.0    NaN      NaN     NaN
2023-11-03   0.0    NaN      NaN     NaN
1.2399999999999989


# 양매도, 양매수에 스트래들

실제 변동

In [27]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i]
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트랭글
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost # 9시 값 합
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue # 15시 값 합
            VKOSPI_Label['profit'].loc[d] = cost - revenue            
        except:     
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan 
            VKOSPI_Label['profit'].loc[d] = np.nan 
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
            VKOSPI_Label['profit'].loc[d] = revenue - cost
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
            VKOSPI_Label['profit'].loc[d] = np.nan

In [28]:
VKOSPI_Label

,VKOSPI_Label,cost,revenue,profit
날짜,,,,
2023-10-10,1,NaN,NaN,NaN
2023-10-11,0,NaN,NaN,NaN
2023-10-12,0,NaN,NaN,NaN
2023-10-13,0,10.82,10.62,0.20
2023-10-16,1,10.66,10.66,0.00
2023-10-17,0,10.74,11.15,-0.41
2023-10-18,0,10.28,10.55,-0.27
2023-10-19,1,10.44,10.85,0.41
2023-10-20,1,11.39,11.35,-0.04


In [29]:
print(VKOSPI_Label['profit'].sum())

0.1999999999999975


In [ ]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i] # 시점
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
            VKOSPI_Label['profit'].loc[d] = revenue - cost 
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
            VKOSPI_Label['profit'].loc[d] = cost - revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
            VKOSPI_Label['profit'].loc[d] = np.nan
# VKOSPI_Label['profit'] = VKOSPI_Label['revenue'] - VKOSPI_Label['cost']
print(VKOSPI_Label)
print(VKOSPI_Label['profit'].sum())

            VKOSPI_Label   cost  revenue  profit
2023-10-10           1.0    NaN      NaN     NaN
2023-10-11           1.0    NaN      NaN     NaN
2023-10-12           1.0    NaN      NaN     NaN
2023-10-13           0.0  10.82    10.62   -0.20
2023-10-16           0.0  10.66    10.66    0.00
2023-10-17           0.0  10.74    11.15    0.41
2023-10-18           1.0  10.28    10.55   -0.27
2023-10-19           0.0  10.44    10.85    0.41
2023-10-20           0.0  11.39    11.35   -0.04
2023-10-23           1.0  10.60    10.29    0.31
2023-10-24           1.0  10.08    10.44   -0.36
2023-10-25           0.0   9.35     9.08   -0.27
2023-10-26           0.0   9.63    10.86    1.23
2023-10-27           1.0   9.39     9.14    0.25
2023-10-30           1.0   8.71     8.59    0.12
2023-10-31           1.0   8.20     7.73    0.47
2023-11-01           0.0    NaN      NaN   -0.47
2023-11-02           1.0    NaN      NaN     NaN
2023-11-03           0.0    NaN      NaN   -0.47
1.1199999999999957


LogisticRegression

In [ ]:
for i in range(len(LogisticRegression.index)):
    d = LogisticRegression.index[i]
    if LogisticRegression.loc[d, 'LogisticRegression'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
LogisticRegression['profit'] = LogisticRegression['revenue'] - LogisticRegression['cost']
print(LogisticRegression)
print(LogisticRegression['profit'].sum())

            LogisticRegression   cost  revenue  profit
2023-10-10                 0.0    NaN      NaN     NaN
2023-10-11                 0.0    NaN      NaN     NaN
2023-10-12                 0.0    NaN      NaN     NaN
2023-10-13                 0.0  10.82    10.62   -0.20
2023-10-16                 0.0  10.66    10.66    0.00
2023-10-17                 1.0  10.74    11.15    0.41
2023-10-18                 0.0  10.28    10.55    0.27
2023-10-19                 0.0  10.44    10.85    0.41
2023-10-20                 1.0  11.39    11.35   -0.04
2023-10-23                 1.0  10.60    10.29   -0.31
2023-10-24                 0.0  10.08    10.44    0.36
2023-10-25                 0.0   9.35     9.08   -0.27
2023-10-26                 0.0   9.63    10.86    1.23
2023-10-27                 1.0   9.39     9.14   -0.25
2023-10-30                 0.0   8.71     8.59   -0.12
2023-10-31                 0.0   8.20     7.73   -0.47
2023-11-01                 1.0    NaN      NaN     NaN
2023-11-02

DecisionTree

In [ ]:
for i in range(len(DecisionTree.index)):
    d = DecisionTree.index[i] # 시점
    if DecisionTree.loc[d, 'DecisionTree'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
            DecisionTree['profit'].loc[d] = revenue - cost 
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
            DecisionTree['profit'].loc[d] = cost - revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
            DecisionTree['profit'].loc[d] = np.nan
# DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
print(DecisionTree)
print(DecisionTree['profit'].sum())

KeyError: 'profit'

In [ ]:
for i in range(len(DecisionTree.index)):
    d = DecisionTree.index[i]
    if DecisionTree.loc[d, 'DecisionTree'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB09sp['기준행사가'].loc[d]
            cost += PTB09sp['기준행사가'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
print(DecisionTree)
print(DecisionTree['profit'].sum())

            DecisionTree   cost  revenue  profit
2023-10-10           0.0    NaN      NaN     NaN
2023-10-11           0.0    NaN      NaN     NaN
2023-10-12           0.0    NaN      NaN     NaN
2023-10-13           0.0  10.82    10.62   -0.20
2023-10-16           0.0  10.66    10.66    0.00
2023-10-17           1.0  10.74    11.15    0.41
2023-10-18           0.0  10.28    10.55    0.27
2023-10-19           0.0  10.44    10.85    0.41
2023-10-20           1.0  11.39    11.35   -0.04
2023-10-23           0.0  10.60    10.29   -0.31
2023-10-24           1.0  10.08    10.44    0.36
2023-10-25           0.0   9.35     9.08   -0.27
2023-10-26           1.0   9.63    10.86    1.23
2023-10-27           1.0   9.39     9.14   -0.25
2023-10-30           1.0   8.71     8.59   -0.12
2023-10-31           0.0   8.20     7.73   -0.47
2023-11-01           1.0    NaN      NaN     NaN
2023-11-02           0.0    NaN      NaN     NaN
2023-11-03           0.0    NaN      NaN     NaN
1.0200000000000014


SVC

In [ ]:
for i in range(len(SVC.index)):
    d = SVC.index[i]
    if SVC.loc[d, 'SVC'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
SVC['profit'] = SVC['revenue'] - SVC['cost']
print(SVC)
print(SVC['profit'].sum())

            SVC   cost  revenue  profit
2023-10-10  0.0    NaN      NaN     NaN
2023-10-11  0.0    NaN      NaN     NaN
2023-10-12  0.0    NaN      NaN     NaN
2023-10-13  0.0  10.82    10.62   -0.20
2023-10-16  0.0  10.66    10.66    0.00
2023-10-17  1.0  10.74    11.15    0.41
2023-10-18  0.0  10.28    10.55    0.27
2023-10-19  0.0  10.44    10.85    0.41
2023-10-20  1.0  11.39    11.35   -0.04
2023-10-23  1.0  10.60    10.29   -0.31
2023-10-24  0.0  10.08    10.44    0.36
2023-10-25  0.0   9.35     9.08   -0.27
2023-10-26  0.0   9.63    10.86    1.23
2023-10-27  1.0   9.39     9.14   -0.25
2023-10-30  0.0   8.71     8.59   -0.12
2023-10-31  0.0   8.20     7.73   -0.47
2023-11-01  0.0    NaN      NaN     NaN
2023-11-02  0.0    NaN      NaN     NaN
2023-11-03  0.0    NaN      NaN     NaN
1.0200000000000014


RandomForest

In [ ]:
for i in range(len(RandomForest.index)):
    d = RandomForest.index[i]
    if RandomForest.loc[d, 'RandomForest'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
RandomForest['profit'] = RandomForest['revenue'] - RandomForest['cost']
print(RandomForest)
print(RandomForest['profit'].sum())

            RandomForest   cost  revenue  profit
2023-10-10           0.0    NaN      NaN     NaN
2023-10-11           0.0    NaN      NaN     NaN
2023-10-12           0.0    NaN      NaN     NaN
2023-10-13           0.0  10.82    10.62   -0.20
2023-10-16           0.0  10.66    10.66    0.00
2023-10-17           1.0  10.74    11.15    0.41
2023-10-18           0.0  10.28    10.55    0.27
2023-10-19           0.0  10.44    10.85    0.41
2023-10-20           1.0  11.39    11.35   -0.04
2023-10-23           1.0  10.60    10.29   -0.31
2023-10-24           0.0  10.08    10.44    0.36
2023-10-25           0.0   9.35     9.08   -0.27
2023-10-26           0.0   9.63    10.86    1.23
2023-10-27           1.0   9.39     9.14   -0.25
2023-10-30           0.0   8.71     8.59   -0.12
2023-10-31           0.0   8.20     7.73   -0.47
2023-11-01           1.0    NaN      NaN     NaN
2023-11-02           0.0    NaN      NaN     NaN
2023-11-03           0.0    NaN      NaN     NaN
1.0200000000000014


XGB

In [ ]:
for i in range(len(XGB.index)):
    d = XGB.index[i]
    if XGB.loc[d, 'XGB'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
XGB['profit'] = XGB['revenue'] - XGB['cost']
print(XGB)
print(XGB['profit'].sum())

            XGB   cost  revenue  profit
2023-10-10    0    NaN      NaN     NaN
2023-10-11    0    NaN      NaN     NaN
2023-10-12    0    NaN      NaN     NaN
2023-10-13    0  10.82    10.62   -0.20
2023-10-16    1  10.66    10.66    0.00
2023-10-17    0  10.74    11.15    0.41
2023-10-18    0  10.28    10.55    0.27
2023-10-19    0  10.44    10.85    0.41
2023-10-20    1  11.39    11.35   -0.04
2023-10-23    1  10.60    10.29   -0.31
2023-10-24    0  10.08    10.44    0.36
2023-10-25    0   9.35     9.08   -0.27
2023-10-26    1   9.63    10.86    1.23
2023-10-27    1   9.39     9.14   -0.25
2023-10-30    0   8.71     8.59   -0.12
2023-10-31    0   8.20     7.73   -0.47
2023-11-01    1    NaN      NaN     NaN
2023-11-02    0    NaN      NaN     NaN
2023-11-03    0    NaN      NaN     NaN
1.0200000000000014


LGBM

In [ ]:
for i in range(len(LGBM.index)):
    d = LGBM.index[i]
    if LGBM.loc[d, 'LGBM'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['기준행사가'].loc[d]
            cost += PTB9sp['기준행사가'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['기준행사가'].loc[d]
            revenue += PTB15sp['기준행사가'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
LGBM['profit'] = LGBM['revenue'] - LGBM['cost']
print(LGBM)
print(LGBM['profit'].sum())

            LGBM   cost  revenue  profit
2023-10-10   0.0    NaN      NaN     NaN
2023-10-11   0.0    NaN      NaN     NaN
2023-10-12   0.0    NaN      NaN     NaN
2023-10-13   0.0  10.82    10.62   -0.20
2023-10-16   1.0  10.66    10.66    0.00
2023-10-17   0.0  10.74    11.15    0.41
2023-10-18   0.0  10.28    10.55    0.27
2023-10-19   0.0  10.44    10.85    0.41
2023-10-20   1.0  11.39    11.35   -0.04
2023-10-23   1.0  10.60    10.29   -0.31
2023-10-24   0.0  10.08    10.44    0.36
2023-10-25   0.0   9.35     9.08   -0.27
2023-10-26   1.0   9.63    10.86    1.23
2023-10-27   1.0   9.39     9.14   -0.25
2023-10-30   0.0   8.71     8.59   -0.12
2023-10-31   0.0   8.20     7.73   -0.47
2023-11-01   0.0    NaN      NaN     NaN
2023-11-02   0.0    NaN      NaN     NaN
2023-11-03   0.0    NaN      NaN     NaN
1.0200000000000014


# 양매도, 양매수 스트랭글

실제 변동

In [ ]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i] # 시점
    if VKOSPI_Label.loc[d, 'VKOSPI_Label'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            VKOSPI_Label['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            VKOSPI_Label['revenue'].loc[d] = revenue
        except:
            VKOSPI_Label['cost'].loc[d] = np.nan
            VKOSPI_Label['revenue'].loc[d] = np.nan
VKOSPI_Label['profit'] = VKOSPI_Label['revenue'] - VKOSPI_Label['cost']
print(VKOSPI_Label)
print(VKOSPI_Label['profit'].sum())

            VKOSPI_Label  cost  revenue  profit
2023-10-10           1.0   NaN      NaN     NaN
2023-10-11           1.0   NaN      NaN     NaN
2023-10-12           1.0   NaN      NaN     NaN
2023-10-13           0.0  3.74     3.62   -0.12
2023-10-16           0.0  3.57     3.60    0.03
2023-10-17           0.0  3.59     3.79    0.20
2023-10-18           1.0  3.20     3.34    0.14
2023-10-19           0.0  3.52     3.87    0.35
2023-10-20           0.0  4.31     4.25   -0.06
2023-10-23           1.0  3.54     3.36   -0.18
2023-10-24           1.0  3.09     3.25    0.16
2023-10-25           0.0  2.56     2.44   -0.12
2023-10-26           0.0  2.94     3.95    1.01
2023-10-27           1.0   NaN      NaN     NaN
2023-10-30           1.0   NaN      NaN     NaN
2023-10-31           1.0   NaN      NaN     NaN
2023-11-01           0.0   NaN      NaN     NaN
2023-11-02           1.0   NaN      NaN     NaN
2023-11-03           0.0   NaN      NaN     NaN
1.4099999999999988


LogisticRegression

In [ ]:
for i in range(len(LogisticRegression.index)):
    d = LogisticRegression.index[i]
    if LogisticRegression.loc[d, 'LogisticRegression'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LogisticRegression['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LogisticRegression['revenue'].loc[d] = revenue
        except:
            LogisticRegression['cost'].loc[d] = np.nan
            LogisticRegression['revenue'].loc[d] = np.nan
LogisticRegression['profit'] = LogisticRegression['revenue'] - LogisticRegression['cost']
print(LogisticRegression)
print(LogisticRegression['profit'].sum())

            LogisticRegression  cost  revenue  profit
2023-10-10                 0.0   NaN      NaN     NaN
2023-10-11                 0.0   NaN      NaN     NaN
2023-10-12                 0.0   NaN      NaN     NaN
2023-10-13                 0.0  3.74     3.62   -0.12
2023-10-16                 0.0  3.57     3.60    0.03
2023-10-17                 1.0  3.59     3.79    0.20
2023-10-18                 0.0  3.20     3.34    0.14
2023-10-19                 0.0  3.52     3.87    0.35
2023-10-20                 1.0  4.31     4.25   -0.06
2023-10-23                 1.0  3.54     3.36   -0.18
2023-10-24                 0.0  3.09     3.25    0.16
2023-10-25                 0.0  2.56     2.44   -0.12
2023-10-26                 0.0  2.94     3.95    1.01
2023-10-27                 1.0   NaN      NaN     NaN
2023-10-30                 0.0   NaN      NaN     NaN
2023-10-31                 0.0   NaN      NaN     NaN
2023-11-01                 1.0   NaN      NaN     NaN
2023-11-02                 0

DecisionTree

In [ ]:
for i in range(len(DecisionTree.index)):
    d = DecisionTree.index[i]
    if DecisionTree.loc[d, 'DecisionTree'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            DecisionTree['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            DecisionTree['revenue'].loc[d] = revenue
        except:
            DecisionTree['cost'].loc[d] = np.nan
            DecisionTree['revenue'].loc[d] = np.nan
DecisionTree['profit'] = DecisionTree['revenue'] - DecisionTree['cost']
print(DecisionTree)
print(DecisionTree['profit'].sum())

            DecisionTree  cost  revenue  profit
2023-10-10           0.0   NaN      NaN     NaN
2023-10-11           0.0   NaN      NaN     NaN
2023-10-12           0.0   NaN      NaN     NaN
2023-10-13           0.0  3.74     3.62   -0.12
2023-10-16           0.0  3.57     3.60    0.03
2023-10-17           1.0  3.59     3.79    0.20
2023-10-18           0.0  3.20     3.34    0.14
2023-10-19           0.0  3.52     3.87    0.35
2023-10-20           1.0  4.31     4.25   -0.06
2023-10-23           0.0  3.54     3.36   -0.18
2023-10-24           1.0  3.09     3.25    0.16
2023-10-25           0.0  2.56     2.44   -0.12
2023-10-26           1.0  2.94     3.95    1.01
2023-10-27           1.0   NaN      NaN     NaN
2023-10-30           1.0   NaN      NaN     NaN
2023-10-31           0.0   NaN      NaN     NaN
2023-11-01           1.0   NaN      NaN     NaN
2023-11-02           0.0   NaN      NaN     NaN
2023-11-03           0.0   NaN      NaN     NaN
1.4099999999999988


SVC

In [ ]:
for i in range(len(SVC.index)):
    d = SVC.index[i]
    if SVC.loc[d, 'SVC'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            SVC['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            SVC['revenue'].loc[d] = revenue
        except:
            SVC['cost'].loc[d] = np.nan
            SVC['revenue'].loc[d] = np.nan
SVC['profit'] = SVC['revenue'] - SVC['cost']
print(SVC)
print(SVC['profit'].sum())

            SVC  cost  revenue  profit
2023-10-10  0.0   NaN      NaN     NaN
2023-10-11  0.0   NaN      NaN     NaN
2023-10-12  0.0   NaN      NaN     NaN
2023-10-13  0.0  3.74     3.62   -0.12
2023-10-16  0.0  3.57     3.60    0.03
2023-10-17  1.0  3.59     3.79    0.20
2023-10-18  0.0  3.20     3.34    0.14
2023-10-19  0.0  3.52     3.87    0.35
2023-10-20  1.0  4.31     4.25   -0.06
2023-10-23  1.0  3.54     3.36   -0.18
2023-10-24  0.0  3.09     3.25    0.16
2023-10-25  0.0  2.56     2.44   -0.12
2023-10-26  0.0  2.94     3.95    1.01
2023-10-27  1.0   NaN      NaN     NaN
2023-10-30  0.0   NaN      NaN     NaN
2023-10-31  0.0   NaN      NaN     NaN
2023-11-01  0.0   NaN      NaN     NaN
2023-11-02  0.0   NaN      NaN     NaN
2023-11-03  0.0   NaN      NaN     NaN
1.4099999999999988


RandomForest

In [ ]:
for i in range(len(RandomForest.index)):
    d = RandomForest.index[i]
    if RandomForest.loc[d, 'RandomForest'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            RandomForest['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            RandomForest['revenue'].loc[d] = revenue
        except:
            RandomForest['cost'].loc[d] = np.nan
            RandomForest['revenue'].loc[d] = np.nan
RandomForest['profit'] = RandomForest['revenue'] - RandomForest['cost']
print(RandomForest)
print(RandomForest['profit'].sum())

            RandomForest  cost  revenue  profit
2023-10-10           0.0   NaN      NaN     NaN
2023-10-11           0.0   NaN      NaN     NaN
2023-10-12           0.0   NaN      NaN     NaN
2023-10-13           0.0  3.74     3.62   -0.12
2023-10-16           0.0  3.57     3.60    0.03
2023-10-17           1.0  3.59     3.79    0.20
2023-10-18           0.0  3.20     3.34    0.14
2023-10-19           0.0  3.52     3.87    0.35
2023-10-20           1.0  4.31     4.25   -0.06
2023-10-23           1.0  3.54     3.36   -0.18
2023-10-24           0.0  3.09     3.25    0.16
2023-10-25           0.0  2.56     2.44   -0.12
2023-10-26           0.0  2.94     3.95    1.01
2023-10-27           1.0   NaN      NaN     NaN
2023-10-30           0.0   NaN      NaN     NaN
2023-10-31           0.0   NaN      NaN     NaN
2023-11-01           1.0   NaN      NaN     NaN
2023-11-02           0.0   NaN      NaN     NaN
2023-11-03           0.0   NaN      NaN     NaN
1.4099999999999988


XGB

In [ ]:
for i in range(len(XGB.index)):
    d = XGB.index[i]
    if XGB.loc[d, 'XGB'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            XGB['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            XGB['revenue'].loc[d] = revenue
        except:
            XGB['cost'].loc[d] = np.nan
            XGB['revenue'].loc[d] = np.nan
XGB['profit'] = XGB['revenue'] - XGB['cost']
print(XGB)
print(XGB['profit'].sum())

            XGB  cost  revenue  profit
2023-10-10    0   NaN      NaN     NaN
2023-10-11    0   NaN      NaN     NaN
2023-10-12    0   NaN      NaN     NaN
2023-10-13    0  3.74     3.62   -0.12
2023-10-16    1  3.57     3.60    0.03
2023-10-17    0  3.59     3.79    0.20
2023-10-18    0  3.20     3.34    0.14
2023-10-19    0  3.52     3.87    0.35
2023-10-20    1  4.31     4.25   -0.06
2023-10-23    1  3.54     3.36   -0.18
2023-10-24    0  3.09     3.25    0.16
2023-10-25    0  2.56     2.44   -0.12
2023-10-26    1  2.94     3.95    1.01
2023-10-27    1   NaN      NaN     NaN
2023-10-30    0   NaN      NaN     NaN
2023-10-31    0   NaN      NaN     NaN
2023-11-01    1   NaN      NaN     NaN
2023-11-02    0   NaN      NaN     NaN
2023-11-03    0   NaN      NaN     NaN
1.4099999999999988


LGBM

In [ ]:
for i in range(len(LGBM.index)):
    d = LGBM.index[i]
    if LGBM.loc[d, 'LGBM'] == 0: # 변동성 하락 양매도 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
    else: # label == 1 변동성 상승 양매수 스트래들
        try:
            cost = 0
            cost += CTB9sp['S+4'].loc[d]
            cost += PTB9sp['S-4'].loc[d]
            LGBM['cost'].loc[d] = cost
            revenue = 0
            revenue += CTB15sp['S+4'].loc[d]
            revenue += PTB15sp['S-4'].loc[d]
            LGBM['revenue'].loc[d] = revenue
        except:
            LGBM['cost'].loc[d] = np.nan
            LGBM['revenue'].loc[d] = np.nan
LGBM['profit'] = LGBM['revenue'] - LGBM['cost']
print(LGBM)
print(LGBM['profit'].sum())

            LGBM  cost  revenue  profit
2023-10-10   0.0   NaN      NaN     NaN
2023-10-11   0.0   NaN      NaN     NaN
2023-10-12   0.0   NaN      NaN     NaN
2023-10-13   0.0  3.74     3.62   -0.12
2023-10-16   1.0  3.57     3.60    0.03
2023-10-17   0.0  3.59     3.79    0.20
2023-10-18   0.0  3.20     3.34    0.14
2023-10-19   0.0  3.52     3.87    0.35
2023-10-20   1.0  4.31     4.25   -0.06
2023-10-23   1.0  3.54     3.36   -0.18
2023-10-24   0.0  3.09     3.25    0.16
2023-10-25   0.0  2.56     2.44   -0.12
2023-10-26   1.0  2.94     3.95    1.01
2023-10-27   1.0   NaN      NaN     NaN
2023-10-30   0.0   NaN      NaN     NaN
2023-10-31   0.0   NaN      NaN     NaN
2023-11-01   0.0   NaN      NaN     NaN
2023-11-02   0.0   NaN      NaN     NaN
2023-11-03   0.0   NaN      NaN     NaN
1.4099999999999988
